In [28]:
import time
import pandas as pd
from datetime import datetime
import joblib
import os.path

# Firebase Libraries
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

# Machine Learning Libraries
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn import metrics
from sklearn import datasets

# GUI Libraries
import tkinter as tk

# Fetch the service account key JSON file contents
cred = credentials.Certificate("serviceAccountKey.json")

# Initialize the app if not already
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://smartyblinds-default-rtdb.firebaseio.com/'
    })

output_message = " "

# Set the initial value of SERIAL_NUM
SERIAL_NUM = 123 

tree_type = "Time.sav"

def predict_function(pred):
    if(os.path.isfile(tree_type)):
        
        # Retrieve Model
        dtree = joblib.load(tree_type)

        # Get Current sensor data
        ref_event = db.reference('Blinds/' + str(SERIAL_NUM) + '/Current/Light')
        current_light = ref_event.get()

        ref_event = db.reference('Blinds/' + str(SERIAL_NUM) + '/Current/Temp')
        current_temperature = ref_event.get()

        ref_event = db.reference('Blinds/' + str(SERIAL_NUM) + '/Current/RTC')
        current_RTC = ref_event.get()
        dateOBJ = datetime.strptime(str(current_RTC), "%Y-%m-%d %H:%M:%S")

        day = dateOBJ.weekday()

        month = dateOBJ.month

        minutes = (dateOBJ.hour * 60) + (dateOBJ.minute)

        print("Light, Temp, Day, Hour")
        print("0-Mon, 1-Tue, 2-Wed, 3-Thu, 4-Fri, 5_Sat, 6-Sun")

        current_light, current_temperature, day, minutes = pred.split()

        current_test = [[current_light,current_temperature,day,int(minutes)*60]]

        print(current_test)

        #Current sensor data is tested with the model
        new_output = int(dtree.predict(current_test))
        
        print(new_output)

        if new_output == 1:
            print("blinds rolled up: " + str(new_output))
            output_message = "blinds rolled up: " + str(new_output)
        else:
            print("blinds rolled down " + str(new_output))
            output_message = "blinds rolled down: " + str(new_output)

        ref_event = db.reference('Blinds/' + str(SERIAL_NUM) + '/Blind_State')
        blind_event = ref_event.get()

        if (new_output != blind_event):

            #control blinds automatically
            ref = db.reference('Blinds/' + str(SERIAL_NUM) + '/')
            ref.update({"Operation": "Auto"})
            ref.update({"Blind_State": new_output})
            
        return output_message
    
# Create the GUI window
def create_gui():
    
    def add_log_message(message):
        message = entry.get()
        ouptut_msg = predict_function(message)
        if message: 
            current_time = time.strftime("%Y-%m-%d %H:%M:%S")
            log_text.insert("end", "{} - {} output: {}\n".format(current_time, message,ouptut_msg))
            entry.delete(0, tk.END)
            
    def change_tree(tree):
        global tree_type
        tree_type = tree
        current_time = time.strftime("%Y-%m-%d %H:%M:%S")
        log_text.insert("end", "{} - Tree Type: {}\n".format(current_time, tree_type))
        entry.delete(0, tk.END)
            
    def clear_log_messages():
        log_text.delete("1.0", "end")
    
    root = tk.Tk()

    #text
    label = tk.Label(root, text="Light, Temp, Day, Hour \n 0-Mon, 1-Tue, 2-Wed, 3-Thu, 4-Fri, 5_Sat, 6-Sun")
    label.grid(row=0, column=0, columnspan=2)

    # Create an entry widget to get input from the user
    entry = tk.Entry(root)
    entry.grid(row=1, column=0, padx=10, pady=10)

    # Create a button widget that adds a log message when clicked
    button = tk.Button(root, text="Predict", command=lambda: add_log_message(""))
    button.grid(row=1, column=1, padx=10, pady=10)

    # Create a label widget to display log messages
    log_text = tk.Text(root, wrap="word")
    log_text.grid(row=0, column=2, columnspan=2, rowspan=2,padx=10, pady=10)

    button1 = tk.Button(root, text="Time Model", command=lambda: change_tree("Time.sav"))
    button1.grid(row=2, column=0, columnspan=2, padx=10, pady=10)

    button2 = tk.Button(root, text="Light Model", command=lambda: change_tree("Light.sav"))
    button2.grid(row=3, column=0, columnspan=2,padx=10, pady=10)

    button3 = tk.Button(root, text="Temp Model", command=lambda: change_tree("Temp.sav"))
    button3.grid(row=4, column=0, columnspan=2,padx=10, pady=10)

    # Create a button widget that clears the log messages when clicked
    clear_button = tk.Button(root, text="Clear log messages", command=clear_log_messages)
    clear_button.grid(row=2, column=2, columnspan=2, padx=10, pady=10)

    # Run the GUI window
    root.mainloop()
    
create_gui()

Light, Temp, Day, Hour
0-Mon, 1-Tue, 2-Wed, 3-Thu, 4-Fri, 5_Sat, 6-Sun
[['600', '23', '4', 720]]
0
blinds rolled down 0
Light, Temp, Day, Hour
0-Mon, 1-Tue, 2-Wed, 3-Thu, 4-Fri, 5_Sat, 6-Sun
[['400', '23', '4', 720]]
0
blinds rolled down 0
Light, Temp, Day, Hour
0-Mon, 1-Tue, 2-Wed, 3-Thu, 4-Fri, 5_Sat, 6-Sun
[['600', '23', '4', 1140]]
1
blinds rolled up: 1
